In [52]:
import numpy as np
import pandas as pd
import scipy
from scipy.spatial import distance
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from scipy.special import rel_entr
import math
from scipy.stats import wasserstein_distance

In [76]:
Predicted = pd.read_csv("./SpatialDWLS/Outputs/output_Sim5.csv")
Target=pd.read_csv("./Data/Sim5/target.csv")

In [77]:
Predicted.drop(columns="Unnamed: 0", inplace=True)
Predicted.drop(columns="cell_ID", inplace=True)
#Predicted.drop(columns="prediction.score.max",inplace=True)


In [78]:
Target.drop(columns="Unnamed: 0", inplace=True)

In [79]:
for i in range(len(Target)):
    Target.loc[i,:]=Target.loc[i,:]/Target.loc[i,:].sum()

In [80]:
for i in range(len(Predicted)):
    Predicted.loc[i,:]=Predicted.loc[i,:]/Predicted.loc[i,:].sum()
 

In [81]:
columns_p=[]
for i in range(Predicted.shape[1]):
    columns_p.append(Predicted.columns[i])
    

In [82]:
columns_t=[]
for i in range(Target.shape[1]):
    columns_t.append(Target.columns[i])
    

In [83]:
columns_p=sorted(columns_p)
columns_t=sorted(columns_t)

In [84]:
Predicted=Predicted[columns_p]
Target=Target[columns_t]

In [87]:
Target=Target.to_numpy()
Predicted=Predicted.to_numpy()

## Pearson Correlation

In [88]:
num_celltypes=Predicted.shape[1]
d = np.zeros(num_celltypes)
for i in range(0,num_celltypes):
    p=Predicted[:,i]
    t=Target[:,i]
    prs=(np.corrcoef(p,t,rowvar=False))
    d[i]=prs[0,1]

In [89]:
np.mean(np.nan_to_num(d))

0.7357650449168923

In [90]:
d

array([-0.00267924,  0.85098305,  0.73166051,  0.86537846,  0.69303119,
        0.69939338,  0.87607568,  0.89545746,  0.82354606,  0.87837684,
        0.76660079,  0.76754494,  0.71957645])

## Jenson Shannon Divergence

In [91]:
jns=distance.jensenshannon(Predicted,Target,axis=1)
jns=jns[~np.isinf(jns).squeeze()]
np.mean(jns)

0.32926291604301827

## F1 Score, Precision, Recall

In [92]:
thresholds={0.001,0.01,0.1,0.2,0.3}

In [93]:
d = pd.DataFrame(0, index=np.arange(len(thresholds)), columns=["Precision","Recall","F1-score"])
d.index=thresholds
for t in thresholds:
    Predicted_logical=(Predicted>t).astype(int)
    Target_logical=(Target>t).astype(int)
    d.loc[t,"Precision"]=precision_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"Recall"]=recall_score(Target_logical,Predicted_logical,average="micro")
    d.loc[t,"F1-score"]=f1_score(Target_logical,Predicted_logical,average="micro")

In [94]:
d

,Precision,Recall,F1-score
0.100,0.894213,0.678790,0.771750
0.200,0.821893,0.637448,0.718015
0.300,0.689427,0.607374,0.645805
0.010,0.889120,0.728235,0.800676
0.001,0.876629,0.735063,0.799629


## KL Divergence

In [95]:
scipy.stats.wasserstein_distance(np.reshape(Target,(Target.shape[0]*Target.shape[1])),np.reshape(Predicted,(Predicted.shape[0]*Predicted.shape[1])))

0.026828095090475523

In [96]:
to_be_kept=(Predicted>0)

In [97]:
Predicted=Predicted[to_be_kept]
Target=Target[to_be_kept]

In [98]:
kl=scipy.special.rel_entr(Target,Predicted)

In [99]:
sum(kl)/len(kl)

0.06812147207179173

In [27]:
len(Predicted)

3058

In [1179]:
np.reshape(Target,(Target.shape[0]*Target.shape[1])).shape

(8000,)